## Step 1: Import Packages
Modify the system path and load the corresponding packages and functions 

In [7]:
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
ROOT = str(Path("__file__").resolve().parents[1])
sys.path.append(ROOT)
import torch
import argparse
import os.path as osp
from mmcv import Config
from trademaster.utils import replace_cfg_vals
from trademaster.nets.builder import build_net
from trademaster.environments.builder import build_environment
from trademaster.datasets.builder import build_dataset
from trademaster.agents.builder import build_agent
from trademaster.optimizers.builder import build_optimizer
from trademaster.losses.builder import build_loss
from trademaster.trainers.builder import build_trainer
from trademaster.transition.builder import build_transition

## Step 2: Load Configs
Load default config from the folder `configs/high_frequency_trading/high_frequency_trading_BTC_dqn_dqn_adam_mse.py`

In [8]:
parser = argparse.ArgumentParser(description='Download Alpaca Datasets')
parser.add_argument("--config", default=osp.join(ROOT, "configs", "high_frequency_trading", "high_frequency_trading_BTC_dqn_dqn_adam_mse.py"),
                    help="download datasets config file path")
parser.add_argument("--task_name", type=str, default="train")
parser.add_argument("--test_style", type=str, default='-1')
args = parser.parse_args([])
cfg = Config.fromfile(args.config)
task_name = args.task_name

cfg = replace_cfg_vals(cfg)
# update test style
cfg.data.update({'test_style': args.test_style})


In [9]:
cfg

Config (path: /data1/sunshuo/qml/TradeMaster/TradeMasterReBuild/configs/high_frequency_trading/high_frequency_trading_BTC_dqn_dqn_adam_mse.py): {'data': {'type': 'HighFrequencyTradingDataset', 'data_path': 'data/high_frequency_trading/small_BTC', 'train_path': 'data/high_frequency_trading/small_BTC/train.csv', 'valid_path': 'data/high_frequency_trading/small_BTC/valid.csv', 'test_path': 'data/high_frequency_trading/small_BTC/test.csv', 'test_style_path': 'data/high_frequency_trading/small_BTC/test.csv', 'tech_indicator_list': ['imblance_volume_oe', 'sell_spread_oe', 'buy_spread_oe', 'kmid2', 'bid1_size_n', 'ksft2', 'ma_10', 'ksft', 'kmid', 'ask1_size_n', 'trade_diff', 'qtlu_10', 'qtld_10', 'cntd_10', 'beta_10', 'roc_10', 'bid5_size_n', 'rsv_10', 'imxd_10', 'ask5_size_n', 'ma_30', 'max_10', 'qtlu_30', 'imax_10', 'imin_10', 'min_10', 'qtld_30', 'cntn_10', 'rsv_30', 'cntp_10', 'ma_60', 'max_30', 'qtlu_60', 'qtld_60', 'cntd_30', 'roc_30', 'beta_30', 'bid4_size_n', 'rsv_60', 'ask4_size_n', 

## Step 3: Build Dataset
Build datasets from cfg defined above

In [10]:
dataset = build_dataset(cfg)

In [11]:
dataset

## Step 4: Build Reinforcement Learning Environments
Build environments based on cfg and previously-defined dataset



In [12]:
valid_environment = build_environment(
        cfg, default_args=dict(dataset=dataset, task="valid")
    )
test_environment = build_environment(
    cfg, default_args=dict(dataset=dataset, task="test")
)
cfg.environment = cfg.train_environment
train_environment = build_environment(
    cfg, default_args=dict(dataset=dataset, task="train")
)

valid
test


In [13]:
train_environment

In [14]:
valid_environment

In [15]:
test_environment

## Step 5: Build Net 
Update information about the state and action dimension in the config and create nets and optimizer for DQN


In [16]:
action_dim = train_environment.action_dim
state_dim = train_environment.state_dim

cfg.act.update(dict(action_dim=action_dim, state_dim=state_dim))
act = build_net(cfg.act)
act_optimizer = build_optimizer(cfg, default_args=dict(params=act.parameters()))
if cfg.cri:
    cfg.cri.update(dict(action_dim=action_dim, state_dim=state_dim))
    cri = build_net(cfg.cri)
    cri_optimizer = build_optimizer(cfg, default_args=dict(params=cri.parameters()))
else:
    cri = None
    cri_optimizer = None

## Step 6: Build Loss
Build loss from config

In [17]:
criterion = build_loss(cfg)

## Step 7: Build Agent
Build agent from config and detect device

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
agent = build_agent(
        cfg,
        default_args=dict(
            action_dim=action_dim,
            state_dim=state_dim,
            act=act,
            cri=cri,
            act_optimizer=act_optimizer,
            cri_optimizer=cri_optimizer,
            criterion=criterion,
            device=device,
        ),
    )


## Step 8: Build Trainer
Build trainer from config and create work directionary to save the result, model and config

In [20]:

trainer = build_trainer(cfg, default_args=dict(train_environment=train_environment,
                                                valid_environment=valid_environment,
                                                test_environment=test_environment,
                                                agent=agent,
                                                device=device))

cfg.dump(osp.join(ROOT, cfg.work_dir, osp.basename(args.config)))

| Arguments Keep work_dir: /data1/sunshuo/qml/TradeMaster/TradeMasterReBuild/work_dir/high_frequency_trading_BTC_high_frequency_trading_dqn_ddqn_adam_mse


## Step 9: Train the Trainer
Train the trainer based on the config and get results from workdir

In [21]:
if task_name.startswith("train"):
    trainer.train_and_valid()
    trainer.test()
    print("train end")
elif task_name.startswith("test"):
    trainer.test()
    print("test end")


the holding could not be bought all clear
the holding could not be sell all clear
the holding could not be sell all clear
Train Episode: [1/10]
the holding could not be sell all clear
the holding could not be sell all clear
the holding could not be sell all clear
the holding could not be sell all clear
the holding could not be sell all clear
the holding could not be sell all clear
the holding could not be sell all clear
the holding could not be bought all clear
the holding could not be bought all clear
the holding could not be bought all clear
the holding could not be bought all clear
the holding could not be bought all clear
the holding could not be sell all clear
the holding could not be bought all clear
the holding could not be bought all clear
the holding could not be sell all clear
the holding could not be bought all clear
the holding could not be bought all clear
the holding could not be sell all clear
the holding could not be sell all clear
the holding could not be sell all clea